### Data Wrangling

In [1]:
# all modules
import warnings
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
# global stuff
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_rows', 13)
pd.set_option('display.max_columns', 13)


Bad key "text.kerning_factor" on line 4 in
/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
# load dataset
jobs_db = pd.read_csv(r'../data/processed/jobs_db.csv')

In [4]:
api_data=pd.read_json(r'../data/processed/api_data.json')
api_data

,normalized_job_title,parent_uuid,suggestion,uuid
0,data operations director,bb4e5066e69e4228172b4850d2a920f4,Data Operations Director,8fd068c8d9be73abfa678856177b6c40
1,data processing manager,bb4e5066e69e4228172b4850d2a920f4,Data Processing Manager,ac47656fd51c2cd8037057262c910dc4
10,database engineer,596226d5f9ef63de41a852826d483d19,Database Engineer,16b67d7f57698455aefbc1ae1b358b50
100,data conversion operator,78e0562032df97cc82fcc48388cdf34b,Data Conversion Operator,04b0eaf624b2deb45ccd62cf6ec7c7bc
101,data processing control clerk,9b44dbc6767274d8b3a820e133803d00,Data Processing Control Clerk,2616d64becb4f2c478dd42f501bbfbff
102,maintenance data analyst,9b44dbc6767274d8b3a820e133803d00,Maintenance Data Analyst,25fe195cbf915c0824fa6c44e6d0008c
...,...,...,...,...
94,health data analyst,db06efbaa9cb31d42e02c047ffb0a15a,Health Data Analyst,6cf4d336112984368f83181ac5ffe948
95,medical data analyst,db06efbaa9cb31d42e02c047ffb0a15a,Medical Data Analyst,7acaed29536648e45ea2b82d45047fcf
96,data processing services sales representative,96fe8c70b3c2f9ca197722fe88c5d6ca,Data Processing Services Sales Representative,f670246beda8187fbe875401e7bc4ba6


In [5]:
url='https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html=requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
countries = soup.find_all('td')
lista=[]
lista=[i.text for i in countries]
cleaned=[i.strip() for i in lista if len(i)>3]
split=2
country_codes=[cleaned[i:i+split] for i in range(0,len(cleaned),split)]

In [6]:
country_codes=pd.DataFrame(country_codes)
country_codes=country_codes.rename(columns={0:'country',1:'country_code'})
country_codes['country_code']=country_codes['country_code'].str.replace(')','')
country_codes['country_code']=country_codes['country_code'].str.replace('(','')
country_codes

,country,country_code
0,Belgium,BE
1,Greece,EL
2,Lithuania,LT
3,Portugal,PT
4,Bulgaria,BG
5,Spain,ES
...,...,...
66,New Zealand,NZ
67,Taiwan,TW
68,Canada,CA


In [7]:
#Rename columns
jobs_db=jobs_db.rename(columns={'uuid':'person_id','dem_education_level':'ed_level','dem_full_time_job':'job_type','question_bbi_2016wave4_basicincome_awareness':'bi_awareness','question_bbi_2016wave4_basicincome_vote':'bi_vote','question_bbi_2016wave4_basicincome_effect':'bi_effect','question_bbi_2016wave4_basicincome_argumentsfor':'bi_argsfor','question_bbi_2016wave4_basicincome_argumentsagainst':'bi_argsagsagainst'})
jobs_db

,person_id,country_code,rural,uuid.1,ed_level,job_type,...,uuid.3,bi_awareness,bi_vote,bi_effect,bi_argsfor,bi_argsagsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,...,f6e7ee00-deac-0133-4de8-0a81e8b09a82,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,...,54f0f1c0-dda1-0133-a559-0a81e8b09a82,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city,83127080-da3d-0133-c74f-0a81e8b09a82,NaN,no,...,83127080-da3d-0133-c74f-0a81e8b09a82,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,...,15626d40-db13-0133-ea5c-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,...,24954a70-db98-0133-4a64-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
5,28583060-dbe2-0133-ff9c-0a81e8b09a82,AT,countryside,28583060-dbe2-0133-ff9c-0a81e8b09a82,high,yes,...,28583060-dbe2-0133-ff9c-0a81e8b09a82,I understand it fully,I would vote against it,A basic income would not affect my work choices,It encourages financial independence and self-...,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,7d3c5860-d8e2-0133-b635-0a81e8b09a82,SK,city,7d3c5860-d8e2-0133-b635-0a81e8b09a82,medium,no,...,7d3c5860-d8e2-0133-b635-0a81e8b09a82,I have never heard of it,I would probably vote for it,A basic income would not affect my work choices,"It increases solidarity, because it is funded ...",It might encourage people to stop working | It...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,urban,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,...,7d1ac020-dcb4-0133-817a-0a81e8b09a82,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,urban,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,...,39f989f0-db52-0133-8482-0a81e8b09a82,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...


In [9]:
#Drop unnecessary columns
drop_cols=[i for i in list(jobs_db.columns) if i.startswith('uuid')]
jobs_db=jobs_db.drop(columns=drop_cols)
jobs_db.columns

Index(['person_id', 'country_code', 'rural', 'ed_level', 'job_type',
       'normalized_job_code', 'age', 'gender', 'dem_has_children', 'age_group',
       'bi_awareness', 'bi_vote', 'bi_effect', 'bi_argsfor',
       'bi_argsagsagainst'],
      dtype='object')

In [12]:
jobs_db=jobs_db.rename(columns={'normalized_job_code':'uuid'})

In [13]:
jobs_df=pd.merge(jobs_db,country_codes, left_on='country_code',right_on='country_code', how='inner')
jobs_df.columns

Index(['person_id', 'country_code', 'rural', 'ed_level', 'job_type', 'uuid',
       'age', 'gender', 'dem_has_children', 'age_group', 'bi_awareness',
       'bi_vote', 'bi_effect', 'bi_argsfor', 'bi_argsagsagainst', 'country'],
      dtype='object')

In [14]:
df=pd.merge(jobs_df,api_data, left_on='uuid',right_on='uuid')

In [16]:
df.shape

(4926, 19)

In [ ]:
#CLEANING